In [1]:
# dev of full attack of a model

In [2]:
from __future__ import print_function

In [3]:
import matplotlib as mpl
# set this 'backend' when using jupyter; do this before importing pyplot
mpl.use('nbagg')
import matplotlib.pyplot as plt

In [4]:
import torch
import torch.nn as nn

import numpy as np
from scipy import stats

import sys
sys.path.append('/home/felix/Research/Adversarial Research/FGN---Research/')
import Finite_Gaussian_Network_lib as fgnl
import Finite_Gaussian_Network_lib.fgn_helper_lib as fgnh

In [5]:
!gpustat

countdown  Sat Aug 17 01:04:58 2019
[0] GeForce GTX 1080 | 39'C,   0 % |    12 /  8119 MB |
[1] GeForce GTX 1080 | 38'C,   0 % |   836 /  8118 MB |


In [6]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
use_cuda = True
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")
print("Using device:", device)

# manualy set cuda device
torch.cuda.set_device(0)

CUDA Available:  True
Using device: cuda


In [7]:
# # random seeds
# torch.manual_seed(999)
# np.random.seed(999)

# torch.backends.cudnn.deterministic = True
# torch.cuda.manual_seed_all(999)

In [8]:
batch_size = 192
(mnist_train_loader, mnist_val_loader, mnist_test_loader) = fgnh.mnist_dataloaders(batch_size)

In [9]:
# load new model
# model_path = "../Experiments/sample_models/sample_fgn_model_noisy_eval_p0.27_full.pth"
model_path = "../Experiments/sample_models/sample_classic_model_noisy_eval_p0.65_full.pth"
model = torch.load(model_path)
model.to(device)

Feedforward_Classic_net(
  (hidden_layers): ModuleList(
    (0): Dropout(p=0.03125)
    (1): Linear(in_features=784, out_features=32, bias=True)
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.03125)
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Dropout(p=0.03125)
  )
  (ib): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fl): Linear(in_features=32, out_features=10, bias=True)
)

In [10]:
# test the model
model.eval()
# switch to random eval
try:
    model.set_random_eval(True)
except: 
    pass

# loss func
cross_ent_loss = fgnh.def_classical_cross_ent_loss()

test_res = fgnh.test(model, mnist_train_loader,
                        cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

test_res = fgnh.test(model, mnist_val_loader,
                        cross_ent_loss, pred_func=fgnh.cross_ent_pred_accuracy, verbose=True)

Test set - Average loss: 0.2577, Accuracy: 46105/50000 (92%)
Test set - Average loss: 0.2488, Accuracy: 9228/10000 (92%)


In [11]:
mnist_val_loader.sampler

In [12]:
### start of dev work

In [13]:
def get_dataloader_bounds(dataloader):
    
    # given a data loader, finds the absolute min and max of the data
    # (future: get bounds per dimension? - optimize so only uses first batch, and doesn't go through batch twice?)
    
    mini = None
    maxi = None
    
    # go through the data
    for data, _ in dataloader:
        
        # find batch bounds
        batch_min = float(data.min().detach().cpu().numpy())
        batch_max = float(data.max().detach().cpu().numpy())
        
        # compare 
        if mini==None:
            mini = batch_min
        else:
            mini = min(mini,batch_min)
            
        if maxi==None:
            maxi = batch_max
        else:
            maxi = max(maxi,batch_max)

    # return tuple
    return (mini, maxi)

In [14]:
def get_dataloader_classes(dataloader):
    
    # returns an array with all the classes found in data loader:
    # future: optimize so doesnt go through whole dataset?
    
    classes = set()
    
    for _, batch_classes in dataloader:
        
        classes = classes.union(batch_classes.detach().cpu().numpy())
        
    return list(classes)
        

In [15]:
get_dataloader_classes(mnist_train_loader)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
def full_FGSM_attack(model, dataloader,
                     num_attacks=None, targeted=False, attack_params=None, **kwargs):
    # for a given model and dataset, runs #num_attacks untargeted attacks.
    
   # model: a pytorch model that outputs a raw vector for a N-class prediction
    if not isinstance(model, torch.nn.Module):
        raise TypeError("model is not a pytorch module")
    
    # dataloader: a pytorch dataloader
    if not isinstance(dataloader, torch.utils.data.dataloader.DataLoader):
        raise TypeError("dataloader is not a pytorch dataloader")
    # force dataloader sampler to be random
    if not isinstance(dataloader.sampler, torch.utils.data.sampler.RandomSampler):
        raise TypeError("dataloader sampler is not random")

    # num_attacks: if None, will attack every point in dataloader
    # otherwise will attack the given number, chosen randomly (this is why RandomSampler is required)
    if num_attacks==None:
        num_attacks = len(dataloader.dataset)
        
    #targeted: boolean, wether to do a targeted or untargeted attack
        
    # attack_params: dictionary with the parameters for FGSM attack
    if attack_params==None:
        # make some relatively arbitrary choices 
        if targeted:
            data_classes = get_dataloader_classes(dataloader)
        data_bounds = get_dataloader_bounds(dataloader)
        max_noise = (max(data_bounds)-min(data_bounds))/10.0
        loss_func = nn.CrossEntropyLoss()
        step_size = max_noise/5.0
        steps = 5
        confidence_req = 0.5
    else:
        if targeted:
            try:
                data_classes = attack_params['target_class']
            except:
                data_classes = get_dataloader_classes(dataloader)
        max_noise = attack_params['max_noise']
        loss_func = attack_params['loss_func']
        step_size = attack_params['step_size']
        data_bounds = attack_params['data_bounds']
        steps = attack_params['steps']
        confidence_req = attack_params['confidence_req']
    
    
    ### kwargs    
    # verbose: boolean, used to print training stats
    verbose = kwargs['verbose'] if 'verbose' in kwargs else False

    # attack model
    attack_count = 0
    
    # values to return
    successful_attack_count = 0
    confidence_dist = []
    steps_dist = []
    
    # load a batch
    for batch, classes in dataloader:
        # check if enough attacks
        if attack_count>=num_attacks:
            # exit  for batch, classes in dataloader loop
            break
        
        # traverse the batch 
        for data_point, point_class in zip(batch, classes):
            
            # check if enough attacks
            if attack_count>=num_attacks:
                # exit for data_point, point_class in zip(batch, classes):
                break
                           
            # perform attack
            if targeted:
                # pick random class to attack
                target_class=np.random.choice(data_classes)
                if verbose: print("Attack model with target", target_class)
                adv_data, adv_noise, attack_results = fgnl.FGSM_attack_targeted(model, data_point, target_class, 
                                                                    max_noise, loss_func, step_size,
                                                                    data_bounds, steps, confidence_req,
                                                                    **kwargs)
            else: # untargeted
                if verbose: print("Untargeted attack of model")
                adv_data, adv_noise, attack_results = fgnl.FGSM_attack_untargeted(model, data_point, 
                                                                    max_noise, loss_func, step_size,
                                                                    data_bounds, steps, confidence_req,
                                                                   **kwargs)
            
            
            # saved desired results
            # attack successful?
            if (attack_results['confidence']>=confidence_req):
                successful_attack_count+=1
            # final_confidence distribution
            confidence_dist.append(attack_results['confidence'])
            # number of steps
            steps_dist.append(attack_results['steps'])
            
            # confidence
            print(attack_results)
            
            # increment attack count
            attack_count+=1
            if verbose: print("Attack count:", attack_count)
        
            # go to next in batch
        # go to next batch
        
    # return results
    return (successful_attack_count, confidence_dist, steps_dist)

In [18]:
full_FGSM_attack(model, mnist_val_loader, num_attacks=10, targeted=False, verbose=True)

Untargeted attack of model
Early stopping at step 2 with confidence 0.874156177044:
{'confidence': 0.8741562, 'steps': 2}
Attack count: 1
Untargeted attack of model
{'confidence': -1.0, 'steps': 5}
Attack count: 2
Untargeted attack of model
Early stopping at step 2 with confidence 0.707192540169:
{'confidence': 0.70719254, 'steps': 2}
Attack count: 3
Untargeted attack of model
Early stopping at step 2 with confidence 0.681775271893:
{'confidence': 0.6817753, 'steps': 2}
Attack count: 4
Untargeted attack of model
Early stopping at step 2 with confidence 0.563039898872:
{'confidence': 0.5630399, 'steps': 2}
Attack count: 5
Untargeted attack of model
Early stopping at step 2 with confidence 0.707117676735:
{'confidence': 0.7071177, 'steps': 2}
Attack count: 6
Untargeted attack of model
Early stopping at step 2 with confidence 0.734332680702:
{'confidence': 0.7343327, 'steps': 2}
Attack count: 7
Untargeted attack of model
Early stopping at step 3 with confidence 0.501730442047:
{'confidenc

(9,
 [0.8741562,
  -1.0,
  0.70719254,
  0.6817753,
  0.5630399,
  0.7071177,
  0.7343327,
  0.50173044,
  0.6936825,
  0.72287536],
 [2, 5, 2, 2, 2, 2, 2, 3, 3, 2])